In [2]:
import json
import math 

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from word2vec import *


In [ ]:
no_of_bins = 20

videos = {}
with open('videos.json', 'r') as f: 
    videos = json.load(f)

x = []
y = []

for video in videos.values():
    print(type(video), video)

    description = video["video"]["description"].split() or []
    tags = video["video"].get('tags', []) or []
    labels = [inner["label"] for inner in video["video"]["thumbnail_objects"]] or []

    words = description + tags + labels
    x.append(get_vector(words))

    view = video['video']['view_count']
    sub_count = video['channel']['subscriber_count']

    y.append(int(math.tanh(math.log(view) / math.log(sub_count)) * no_of_bins))

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
# Create an instance of the Random Forest Regression model
model = RandomForestRegressor()

# Train the model on the training data
model.fit(x_train, y_train)


RandomForestRegressor()

In [17]:
# Make predictions on the test data
y_pred = model.predict(x_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

for index in range(len(x_test)):
    if y_test[index] >= 15:
        print(y_pred[index], y_test[index])
    # print(index, y_pred[index], y_test[index])

Mean Squared Error: 0.7013685393258426
13.35 15
